

 Here we are going to do binary classification using Yelp Review Sentiment Dataset ([Kaggle Link](https://www.kaggle.com/ilhamfp31/yelp-review-dataset)).

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tqdm import tqdm
import nltk

In [2]:
print(keras.__version__)

2.4.0


In [3]:
train = pd.read_csv('../input/yelp-review-dataset/yelp_review_polarity_csv/train.csv', names = ['sentiment', 'text'] )
test  = pd.read_csv('../input/yelp-review-dataset/yelp_review_polarity_csv/test.csv',  names = ['sentiment', 'text'] )

#### Here, Negative polarity is class 1, and positive class 2.


In [4]:
train = train[:15000]
train.head()

,sentiment,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


#### Before diving into that, lets do little text cleaning/preprocessing.

In [5]:
# pip install bs4
# from bs4 import BeautifulSoup
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.stem import WordNetLemmatizer 
# import re
# lm=WordNetLemmatizer()


# def ReturnCleanText(text):
#         # change the text into lower case.(Note: in case of social media text, it is good to leave them as it is)
#         text = text.lower()
#         # removing xml tags from tweets
#         text =BeautifulSoup(text, 'lxml').get_text()
#         # removing URLS 
#         text =re.sub('https?://[A-Za-z0-9./]+','',text)
#         # removing words with "@"
#         text =re.sub(r'@[A-Za-z0-9]+','',text)
#         # removing special characters
#         text = re.sub(r"\W+|_", ' ', text)
#         # tokenization of sentences
#         text = word_tokenize(text)
#         # lemmatize the text using WordNetn
#         words = [lm.lemmatize(word) for word in text if word not in set(stopwords.words('english'))]   
#         return " ".join(words)
    
# train['clean_text'] = train['text'].apply(ReturnCleanText)

# LSTM(Long Short Term Memory):
> Long short-term memory is an artificial recurrent neural network architecture used in the field of deep learning. Unlike standard feedforward neural networks, LSTM has feedback connections. It can not only process single data points, but also entire sequences of data.

In [6]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout

In [7]:
# Just for example
max_features = 2000
Encoder = keras.layers.experimental.preprocessing.TextVectorization( max_tokens = max_features)
Encoder.adapt(train['text'].values)

vocab = np.array(Encoder.get_vocabulary())
print(vocab[:20])

example ="This is an example to test the encoder that we just created!"
print(Encoder(example).numpy())
print(" ".join(vocab[Encoder(example).numpy()]))

['' '[UNK]' 'the' 'and' 'i' 'a' 'to' 'was' 'of' 'it' 'is' 'in' 'for'
 'that' 'my' 'this' 'but' 'with' 'they' 'you']
[  15   10   56 1968    6 1913    2    1   13   23   39    1]
this is an example to test the [UNK] that we just [UNK]


In [8]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, )
tokenizer.fit_on_texts(train['text'].values)
X = tokenizer.texts_to_sequences(train['text'].values)
X = pad_sequences(X, padding = 'post' ,maxlen=300)
Y = pd.get_dummies(train['sentiment']).values

vocab_size = len(tokenizer.word_index)+1

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(11250, 300) (11250, 2)
(3750, 300) (3750, 2)


# Training with Keras default Embedding Layer

### Keras Embedding Layer: 

Embedding layers in Keras are trained just like any other layer in your network architecture: they are tuned to minimize the loss function by using the selected optimization method. The major difference with other layers, is that their output is not a mathematical function of the input. Instead the input to the layer is used to index a table with the embedding vectors [1]. However, the underlying automatic differentiation engine has no problem to optimize these vectors to minimize the loss function...

So, you cannot say that the Embedding layer in Keras is doing the same as word2vec [2]. Remember that word2vec refers to a very specific network setup which tries to learn an embedding which captures the semantics of words. With Keras's embedding layer, you are just trying to minimize the loss function, so if for instance you are working with a sentiment classification problem, the learned embedding will probably not capture complete word semantics but just their emotional polarity.

More Here: 
1. https://stats.stackexchange.com/questions/324992/how-the-embedding-layer-is-trained-in-keras-embedding-layer
2. https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work

In [10]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(max_features, embid_dim, input_length = X.shape[1]))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          600000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               439296    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 1,080,578
Trainable params: 1,080,578
Non-trainable params: 0
______________________________________________

In [11]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5
88/88 [==============================] - 14s 111ms/step - loss: 0.6354 - accuracy: 0.6088 - val_loss: 0.3236 - val_accuracy: 0.8656
Epoch 2/5
88/88 [==============================] - 8s 96ms/step - loss: 0.3023 - accuracy: 0.8822 - val_loss: 0.2971 - val_accuracy: 0.8813
Epoch 3/5
88/88 [==============================] - 9s 98ms/step - loss: 0.2204 - accuracy: 0.9199 - val_loss: 0.3031 - val_accuracy: 0.8688
Epoch 4/5
88/88 [==============================] - 8s 96ms/step - loss: 0.1769 - accuracy: 0.9378 - val_loss: 0.3226 - val_accuracy: 0.8589
Epoch 5/5
88/88 [==============================] - 8s 96ms/step - loss: 0.2048 - accuracy: 0.9249 - val_loss: 0.3627 - val_accuracy: 0.8584


# Training with GloVe 300D Embeddings

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

Link: https://nlp.stanford.edu/projects/glove/

In [12]:
from tqdm import tqdm
embedding_vector = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

2196018it [03:51, 9504.08it/s]


In [13]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 37637/37637 [00:00<00:00, 262225.43it/s]


In [14]:
embedding_matrix.shape

(37638, 300)

In [15]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, embid_dim, input_length =X.shape[1], weights = [embedding_matrix] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 300)          11291400  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 11,771,978
Trainable params: 480,578
Non-trainable params: 11,291,400
____________________________________

In [16]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5
88/88 [==============================] - 11s 89ms/step - loss: 0.6140 - accuracy: 0.6549 - val_loss: 0.4212 - val_accuracy: 0.8325
Epoch 2/5
88/88 [==============================] - 7s 79ms/step - loss: 0.4253 - accuracy: 0.8231 - val_loss: 0.3975 - val_accuracy: 0.8341
Epoch 3/5
88/88 [==============================] - 7s 80ms/step - loss: 0.3901 - accuracy: 0.8402 - val_loss: 0.4428 - val_accuracy: 0.8240
Epoch 4/5
88/88 [==============================] - 7s 80ms/step - loss: 0.3635 - accuracy: 0.8533 - val_loss: 0.3380 - val_accuracy: 0.8528
Epoch 5/5
88/88 [==============================] - 7s 81ms/step - loss: 0.3140 - accuracy: 0.8664 - val_loss: 0.3536 - val_accuracy: 0.8568


# Training with Word2Vec Pre-trained and Trained Embeddings
Reference: https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

**Word2Vec** is not a singular algorithm, rather, it is a family of model architectures and optimizations that can be used to learn word embeddings from large datasets.

1. Continuous Bag-of-Words Model which predicts the middle word based on surrounding context words. The context consists of a few words before and after the current (middle) word. This architecture is called a bag-of-words model as the order of words in the context is not important.
2. Continuous Skip-gram Model which predict words within a certain range before and after the current word in the same sentence. A worked example of this is given below.

Know more here:
1. https://jalammar.github.io/illustrated-word2vec/
2. https://www.tensorflow.org/tutorials/text/word2vec

### Training a Word2Vec Embedding from scratch using Gensim library:

In [17]:
sentences =[]
for t in  tqdm(range(len(train['text']))):
    text = nltk.word_tokenize(train['text'][t])
    sentences.append(text)

100%|██████████| 15000/15000 [00:24<00:00, 623.24it/s]


##### sg : Either 0 or 1. Default is 0 or CBOW. One must explicitly define Skip-gram by passing 1.

In [18]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences, size=300, min_count=2, sg = 0 )

TypeError: __init__() got an unexpected keyword argument 'size'

In [19]:
words = list(w2v_model.wv.vocab)
print('Vocabulary size: %d' % len(words))

# save model 
filename = 'embedding_word2vec.txt'
w2v_model.wv.save_word2vec_format(filename, binary=False)

NameError: name 'w2v_model' is not defined

In [20]:
embedding_vector = {}
f = open('./embedding_word2vec.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

FileNotFoundError: [Errno 2] No such file or directory: './embedding_word2vec.txt'

In [21]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value    

100%|██████████| 37637/37637 [00:00<00:00, 1252050.41it/s]


In [22]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, embid_dim, input_length =X.shape[1], weights = [ embedding_matrix] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 300)          11291400  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 130       
Total params: 11,771,978
Trainable params: 480,578
Non-trainable params: 11,291,400
____________________________________

In [23]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 50, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/50
88/88 [==============================] - 11s 87ms/step - loss: 0.6929 - accuracy: 0.5204 - val_loss: 0.6933 - val_accuracy: 0.5040
Epoch 2/50
88/88 [==============================] - 7s 80ms/step - loss: 0.6925 - accuracy: 0.5178 - val_loss: 0.6935 - val_accuracy: 0.5040
Epoch 3/50
88/88 [==============================] - 7s 80ms/step - loss: 0.6927 - accuracy: 0.5150 - val_loss: 0.6937 - val_accuracy: 0.5040
Epoch 4/50
88/88 [==============================] - 7s 81ms/step - loss: 0.6916 - accuracy: 0.5291 - val_loss: 0.6937 - val_accuracy: 0.5040
Epoch 5/50
88/88 [==============================] - 7s 80ms/step - loss: 0.6926 - accuracy: 0.5177 - val_loss: 0.6936 - val_accuracy: 0.5040
Epoch 6/50
88/88 [==============================] - 7s 80ms/step - loss: 0.6923 - accuracy: 0.5210 - val_loss: 0.6938 - val_accuracy: 0.5040
Epoch 7/50
88/88 [==============================] - 7s 81ms/step - loss: 0.6919 - accuracy: 0.5255 - val_loss: 0.6937 - val_accuracy: 0.5040
Epoch 8/50
8

### Using Pretrained Word2Vec Embedding
Reference: https://www.kaggle.com/jaskarancr/word2vec-traditional-models

In [24]:
from gensim.models import KeyedVectors
filename = '../input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin'
w2v_pretrained_model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [25]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    try:
        embedding_value = w2v_pretrained_model[word]
        if embedding_value is not None:
            embedding_matrix[i] = embedding_value         
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),300)       

100%|██████████| 37637/37637 [00:00<00:00, 87979.85it/s]


In [26]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, 300, input_length =300, weights = [embedding_matrix ] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 300)          11291400  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130       
Total params: 11,771,978
Trainable params: 480,578
Non-trainable params: 11,291,400
____________________________________

In [27]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5
88/88 [==============================] - 10s 88ms/step - loss: 0.6205 - accuracy: 0.6361 - val_loss: 0.4557 - val_accuracy: 0.7976
Epoch 2/5
88/88 [==============================] - 7s 80ms/step - loss: 0.4441 - accuracy: 0.8039 - val_loss: 0.3815 - val_accuracy: 0.8408
Epoch 3/5
88/88 [==============================] - 7s 80ms/step - loss: 0.3721 - accuracy: 0.8461 - val_loss: 0.3606 - val_accuracy: 0.8549
Epoch 4/5
88/88 [==============================] - 7s 81ms/step - loss: 0.3995 - accuracy: 0.8295 - val_loss: 0.3585 - val_accuracy: 0.8539
Epoch 5/5
88/88 [==============================] - 7s 80ms/step - loss: 0.3558 - accuracy: 0.8563 - val_loss: 0.3552 - val_accuracy: 0.8565


# Using Pretrained word2vec Embedding with Trainable as True

In [28]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, 300, input_length =300, weights = [embedding_matrix ],
                    trainable = True))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 300)          11291400  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 130       
Total params: 11,771,978
Trainable params: 11,771,978
Non-trainable params: 0
__________________________________________

In [29]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5
88/88 [==============================] - 27s 270ms/step - loss: 0.6183 - accuracy: 0.6472 - val_loss: 0.3572 - val_accuracy: 0.8581
Epoch 2/5
88/88 [==============================] - 24s 272ms/step - loss: 0.4338 - accuracy: 0.8256 - val_loss: 0.3818 - val_accuracy: 0.8315
Epoch 3/5
88/88 [==============================] - 23s 263ms/step - loss: 0.3234 - accuracy: 0.8788 - val_loss: 0.3280 - val_accuracy: 0.8669
Epoch 4/5
88/88 [==============================] - 23s 260ms/step - loss: 0.2564 - accuracy: 0.9073 - val_loss: 0.3454 - val_accuracy: 0.8768
Epoch 5/5
88/88 [==============================] - 24s 272ms/step - loss: 0.2098 - accuracy: 0.9209 - val_loss: 0.4229 - val_accuracy: 0.8541
